## Imports

In [ ]:
import sys
import os 
pkg_path = os.path.abspath(os.path.join(os.getcwd(), os.pardir))
sys.path.append(pkg_path)

In [ ]:
import numpy as np

from pynn_brainscales import brainscales2 as pynn

import shtmbss2.addsrc
from shtmbss2.core import hardware_initialization
from shtmbss2.network import SHTMTotal

## Configuration

In [3]:
alphabet_size = 4
num_neurons_per_symbol = 5

hardware_initialization()

INFO  16:53:57,920  demo_helpers Using setup hxcube7fpga0chip57_1


## Network Initialization

In [4]:
shtm = SHTMTotal(alphabet_size, num_neurons_per_symbol, log_permanence='all')

shtm.init_neurons()
shtm.init_connections(debug=True)
shtm.init_external_input(sequence=['A', 'C'], num_repetitions=1000)
shtm.init_rec_exc()

print(shtm.con_plastic[0].permanence)

Initialized external input for sequence ['A', 'C']
Spike times:
A: [0.04, 0.092, 0.14400000000000002, 0.19600000000000004, 0.24800000000000005, 0.30000000000000004, 0.35200000000000004, 0.404, 0.456, 0.508, 0.56, 0.6120000000000001, 0.6640000000000001, 0.7160000000000002, 0.7680000000000002, 0.8200000000000003, 0.8720000000000003, 0.9240000000000004, 0.9760000000000004, 1.0280000000000005, 1.0800000000000005, 1.1320000000000006, 1.1840000000000006, 1.2360000000000007, 1.2880000000000007, 1.3400000000000007, 1.3920000000000008, 1.4440000000000008, 1.4960000000000009, 1.548000000000001, 1.600000000000001, 1.652000000000001, 1.704000000000001, 1.7560000000000011, 1.8080000000000012, 1.8600000000000012, 1.9120000000000013, 1.9640000000000013, 2.0160000000000013, 2.068000000000001, 2.1200000000000006, 2.172, 2.2239999999999998, 2.2759999999999994, 2.327999999999999, 2.3799999999999986, 2.431999999999998, 2.4839999999999978, 2.5359999999999974, 2.587999999999997, 2.6399999999999966, 2.691999

## Network Emulation & Plotting

In [ ]:
for i in range(1):
    shtm.run(runtime=100, steps=1, plasticity_enabled=True)

Error! Wrong runtime
Running emulation step 1/1
Current time: 0
Starting plasticity calculations
t: 0.04060040600406004,  p: 2.997,  dp: -0.003,  x: 1.0,  z: 0.0
[0.040616 0.043304 0.047016 0.0926   0.095304 0.098984 0.1446   0.147272
 0.15092  0.196592 0.199256 0.202912 0.24864  0.25136  0.255072 0.300632
 0.30336  0.307064 0.352608 0.355288 0.358976 0.4046   0.407312 0.411008
 0.456616 0.459296 0.463024 0.508624 0.51132  0.515024 0.560608 0.56328
 0.566952 0.61264  0.615336 0.619008 0.664648 0.667408 0.6712   0.716616
 0.719328 0.723152 0.768616 0.771296 0.774976 0.820616 0.823312 0.827024
 0.872616 0.875296 0.878944 0.924616 0.927328 0.93104  0.976624 0.97928
 0.982952 1.028648 1.03132  1.035    1.0806   1.083304 1.087104 1.132624
 1.13532  1.139072 1.184616 1.18728  1.191    1.236608 1.23932  1.243008
 1.288608 1.291296 1.295016 1.340616 1.34332  1.346968 1.392656 1.39536
 1.399016 1.4446   1.447336 1.4512   1.4966   1.499296 1.503016 1.548608
 1.551312 1.555016 1.600624 1.603288 1

In [ ]:
%matplotlib inline

shtm.plot_events(neuron_types="all")

In [ ]:
%matplotlib inline

shtm.plot_permanence_diff()

In [ ]:
%matplotlib inline

shtm.plot_permanence()

In [ ]:
%matplotlib inline

shtm.plot_v_exc(alphabet_range=[0])

In [ ]:
%matplotlib inline

shtm.plot_v_exc(alphabet_range=range(1, alphabet_size))